# Lassie as retrieval tool

daemon is running in the backend at END_POINT

In [ ]:
END_POINT = "127.0.0.1:38845"

In [ ]:
from io import BytesIO
import requests
import rasterio
import numpy as np
from matplotlib import colors
from matplotlib import colormaps
import matplotlib.pyplot as plt
def lassie_daemon_get(cid):
    url = f"http://{END_POINT}/ipfs/{cid}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.content
        print(f"Data retrieved successfully")
        return data
    else:
        print(f"Error retrieving data: {response.status_code}")
        return None

def lassie_binary_get(cid):
    import subprocess
    lassie_path = '../../../lassie-all/release/lassie'
    # Start the subprocess with `car x -` command, which reads from stdin and outputs to stdout
    process = subprocess.Popen([lassie_path,'fetch', '-o','-', cid], stdout=subprocess.PIPE)

    # Read the output of the subprocess from stdout
    output = process.stdout.read()

    # Wait for the subprocess to finish
    process.wait()

    # Print the output of the subprocess
    return output


In [ ]:
def load_landsat_band(band: bytes, dtype: np.dtype = np.float32) -> np.ndarray:
    with BytesIO(band) as bytes_io:
        with rasterio.open(bytes_io) as dataset:
            return dataset.read(1).astype(dtype)


In [ ]:
red_band_cid = "QmTgttqUf7PvZgdSoe71j3njeEKk1hC3h22n2sQmety3To"
nir_band_cid = "QmZkWaKSuVhFKtAwNbxSogcT6hXHMksXjhgqLu6AXHSUKq"

In [ ]:
# retrieve using the daemon process
red_band_raw = lassie_daemon_get(red_band_cid)
nir_band_raw = lassie_daemon_get(nir_band_cid)

In [ ]:
# retrieve using the binary executable
red_band_raw = lassie_binary_get(red_band_cid)
nir_band_raw = lassie_binary_get(nir_band_cid)

In [ ]:
len(nir_band_raw)

In [ ]:
# fetched object are in CAR format so need to be extracted with car tool before getting the original content.
def car_extract(band_raw):
    import subprocess
    car_path = '../../../lassie-all/go-car/car'
    # Assume that `band_raw` is a bytes object containing the raw image data

    # Start the subprocess with `car x -` command, which reads from stdin and outputs to stdout
    process = subprocess.Popen([car_path, 'x', '-'], stdin=subprocess.PIPE, stdout=subprocess.PIPE)

    # Write the bytes object to the subprocess stdin
    process.stdin.write(band_raw)

    # Close the subprocess stdin to indicate that no more data will be sent
    process.stdin.close()

    # Read the output of the subprocess from stdout
    output = process.stdout.read()

    # Wait for the subprocess to finish
    process.wait()

    # Print the output of the subprocess
    return output


In [ ]:
red_band = car_extract(red_band_raw)
nir_band = car_extract(nir_band_raw)

red_band_4 = load_landsat_band(red_band)
nir_band_5 = load_landsat_band(nir_band)

In [ ]:
eps = 0.0001 # Avoid divide by zero errors
ndvi = (nir_band_5 - red_band_4) / (nir_band_5 + red_band_4 + eps)

In [ ]:

fig, ax = plt.subplots(figsize=(10, 10))

# Remove the axes
ax.set_xticks([])
ax.set_yticks([])

# Set the color map to the min and max values of NDVI
ndvi_min, ndvi_max = np.min(ndvi), np.max(ndvi)
norm = colors.Normalize(vmin=ndvi_min, vmax=ndvi_max)

# Use the normalization object for the image and the color map
green_color_map = colormaps["Greens"]
green_ndvi = green_color_map(norm(ndvi))

img = ax.imshow(green_ndvi)
ax.set_title("NDVI Plot")

cax = fig.add_axes([0.9, 0.15, 0.03, 0.7])
cb = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=green_color_map), cax=cax, orientation='vertical')
cb.set_label("NDVI")

plt.show()